### Define data input files and manual data inputs

In [97]:
import pandas as pd

# Month of the prediction model (no data will be used past this month, as we are predicting production at a point in time)
# Just for our example in this notebook, we use August (so data up through July)
month = 8

province_splits_filepath = "input_data/province_production_breakouts.csv" # % of production for each crop from each province
production_data_filepath = "input_data/FAOSTAT_data_en_8-7-2024.csv" # total produced amount of each crop for Total Senegal
rainfall_data_filepath = "input_data/rain_adm1_data.csv" # rainfall data by province
vhi_data_filepath = "input_data/vhi_adm1_dekad_data.csv" # vhi data by province
temp_data_filepath = "input_data/temperature_data_monthly_province.csv"

# Values of 2023 for each crop (manually pulled from website)
production_actuals_2023 = pd.DataFrame({
    "Crop": ["Peanut", "Corn", "Millet", "Rice", "Sorghum"],
    "Production": [1728000, 929000, 1353000, 1521000, 445000],
    "Year": [2023, 2023, 2023, 2023, 2023]
})

production_actuals_2024 = pd.DataFrame({
    "Crop": ["Peanut", "Corn", "Millet", "Rice", "Sorghum"],
    "Production": [1728000, 929000, 1353000, 1521000, 445000],
    "Year": [2024, 2024, 2024, 2024, 2024]
})

### Production data pre-processing

In [98]:
import pandas as pd

def load_province_breakouts_and_remove_unneeded_columns(filename):

    # Province crop data - manually brought in from https://ipad.fas.usda.gov/countrysummary/default.aspx?id=SG
    province_splits = pd.read_csv(filename)

    # Remove all columns that have Labeled in the title
    province_splits = province_splits.loc[:, ~province_splits.columns.str.contains('Labeled')]

    # Remove Total_Pct and Missing_Pct and Missing_Provinces columns
    province_splits = province_splits.drop(columns=["Total_Pct", "Missing_Pct", "Missing_Provinces"])

    return province_splits

province_splits = load_province_breakouts_and_remove_unneeded_columns(province_splits_filepath)

province_splits

,Crop,Dakar,Diourbel,Fatick,Kaffrine,Kaolack,Kedougou,Kolda,Louga,Matam,Saint louis,Sedhiou,Tambacounda,Thies,Ziguinchor
0,Corn,0.017778,0.017778,0.10,0.017778,0.16,0.017778,0.24,0.017778,0.017778,0.017778,0.210000,0.13,0.017778,0.017778
1,Cotton,0.000000,0.000000,0.00,0.050000,0.01,0.060000,0.62,0.000000,0.000000,0.000000,0.010000,0.25,0.000000,0.000000
2,Millet,0.002000,0.070000,0.18,0.230000,0.16,0.002000,0.08,0.050000,0.002000,0.002000,0.080000,0.06,0.080000,0.002000
3,Peanut,0.015714,0.015714,0.13,0.220000,0.15,0.015714,0.17,0.080000,0.015714,0.015714,0.015714,0.07,0.070000,0.015714
4,Rice,0.010000,0.010000,0.01,0.010000,0.01,0.010000,0.27,0.010000,0.010000,0.220000,0.250000,0.01,0.010000,0.160000
5,Sorghum,0.020000,0.020000,0.02,0.260000,0.11,0.020000,0.15,0.020000,0.020000,0.020000,0.130000,0.17,0.020000,0.020000


In [99]:
crop_list = ["Peanut", "Corn", "Millet", "Rice", "Sorghum"]
crop_list_production_names = ["Groundnuts, excluding shelled", "Maize (corn)", "Millet", "Rice", "Sorghum"]
element = "Production"

def load_and_filter_production_data(production_data_filepath, crop_list_production_names, element):
    # Load data
    production_data = pd.read_csv(production_data_filepath)

    # Print total rows
    print("Total rows in production data: ", len(production_data))

    # Filter for just production data
    production_data = production_data[production_data["Element"] == element]

    # Filter for just when production is > 0
    production_data = production_data[production_data["Value"] > 0]
    
    # Print that we are filtering for our crop list
    print("Filtering for crops: ", crop_list_production_names)

    # Filter for just the crops we are interested in
    production_data = production_data[production_data["Item"].isin(crop_list_production_names)]

    # Print rows remaining
    print("Rows remaining after filtering: ", len(production_data))

    return production_data

production_data = load_and_filter_production_data(production_data_filepath, crop_list_production_names, element)

# View table
production_data.tail()


Total rows in production data:  7795
Filtering for crops:  ['Groundnuts, excluding shelled', 'Maize (corn)', 'Millet', 'Rice', 'Sorghum']
Rows remaining after filtering:  310


,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
6730,QCL,Crops and livestock products,686,Senegal,5510,Production,0114,Sorghum,2018,2018,t,295463.11,A,Official figure,NaN
6733,QCL,Crops and livestock products,686,Senegal,5510,Production,0114,Sorghum,2019,2019,t,270168.31,A,Official figure,NaN
6736,QCL,Crops and livestock products,686,Senegal,5510,Production,0114,Sorghum,2020,2020,t,377322.86,A,Official figure,NaN
6739,QCL,Crops and livestock products,686,Senegal,5510,Production,0114,Sorghum,2021,2021,t,352474.34,A,Official figure,NaN
6742,QCL,Crops and livestock products,686,Senegal,5510,Production,0114,Sorghum,2022,2022,t,363164.00,A,Official figure,NaN


In [100]:
def change_crop_names_to_match_other_data(production_data, crop_list_production_names, crop_list):
    # Print value counts of each crop
    print("--------Original Names--------")
    print(production_data["Item"].value_counts())
    
    # Change crop names to match other data
    production_data["Item"] = production_data["Item"].replace(crop_list_production_names, crop_list)

    # Print value counts of each crop
    print("--------Cleaned Names--------")
    print(production_data["Item"].value_counts())
    
    return production_data

# Change names
production_data = change_crop_names_to_match_other_data(production_data, crop_list_production_names, crop_list)


--------Original Names--------
Item
Groundnuts, excluding shelled    62
Maize (corn)                     62
Millet                           62
Rice                             62
Sorghum                          62
Name: count, dtype: int64
--------Cleaned Names--------
Item
Peanut     62
Corn       62
Millet     62
Rice       62
Sorghum    62
Name: count, dtype: int64


In [101]:
columns_to_keep = ["Item", "Value", "Year"]
new_column_names = ["Crop", "Production", "Year"]

def remove_unneeded_columns(data, columns_to_keep):
    # Print the columns we are keeping
    print("Keeping only these columns: ", columns_to_keep)
    
    # Only keep needed columns
    data = data[columns_to_keep]

    # Change the column names to new_column_names
    data.columns = new_column_names

    return data

# Only keep Item, Value, Year columns
production_data = remove_unneeded_columns(production_data, columns_to_keep)

# View data
production_data.tail()

Keeping only these columns:  ['Item', 'Value', 'Year']


,Crop,Production,Year
6730,Sorghum,295463.11,2018
6733,Sorghum,270168.31,2019
6736,Sorghum,377322.86,2020
6739,Sorghum,352474.34,2021
6742,Sorghum,363164.00,2022


In [102]:
def append_2023_production_data_manually(production_data, production_actuals_2023):
    # Print year range before appending
    print("Year range before appending manual data: ", production_data["Year"].min(), production_data["Year"].max())

    # Append 2023 data to production (append is no longer used in pandas, so remember to use the new function)
    production_data = pd.concat([production_data, production_actuals_2023], axis=0)

    # Print year range after appending
    print("Year range after appending manual data: ", production_data["Year"].min(), production_data["Year"].max())

    return production_data

# Append 2023 data to production (append is no longer used in pandas, so remember to use the new function)
production_data = append_2023_production_data_manually(production_data, production_actuals_2023)
production_data = append_2023_production_data_manually(production_data, production_actuals_2024)

Year range before appending manual data:  1961 2022
Year range after appending manual data:  1961 2023
Year range before appending manual data:  1961 2023
Year range after appending manual data:  1961 2024


In [103]:
# All production data processing shown together
def process_production_data(production_data_filepath, crop_list_production_names, crop_list, element, columns_to_keep, production_actuals_2023):
    # Print status
    print("######## Processing Production Data ##########")

    # Load and filter production data
    production_data = load_and_filter_production_data(production_data_filepath, crop_list_production_names, element)

    # Change crop names to match other data
    production_data = change_crop_names_to_match_other_data(production_data, crop_list_production_names, crop_list)

    # Only keep Item, Value, Year columns
    production_data = remove_unneeded_columns(production_data, columns_to_keep)

    # Append 2023 data to production data manually
    production_data = append_2023_production_data_manually(production_data, production_actuals_2023)
    production_data = append_2023_production_data_manually(production_data, production_actuals_2024)
    
    print("##############################################")

    return production_data

# Process production data
production_data = process_production_data(production_data_filepath, crop_list_production_names, crop_list, element, columns_to_keep, production_actuals_2023)

# Calculate number of rows for each item in total not by column
print(f"---Final Years of data available for each crop---")
print(production_data['Crop'].value_counts())
print("--------------------------------------------")

# Ensure we have the same amount of data for each crop
assert production_data['Crop'].value_counts().nunique() == 1

# Save processed data in data folder
production_data.to_csv("processed_data/production_data.csv", index=False)

# View data
production_data.tail()

######## Processing Production Data ##########
Total rows in production data:  7795
Filtering for crops:  ['Groundnuts, excluding shelled', 'Maize (corn)', 'Millet', 'Rice', 'Sorghum']
Rows remaining after filtering:  310
--------Original Names--------
Item
Groundnuts, excluding shelled    62
Maize (corn)                     62
Millet                           62
Rice                             62
Sorghum                          62
Name: count, dtype: int64
--------Cleaned Names--------
Item
Peanut     62
Corn       62
Millet     62
Rice       62
Sorghum    62
Name: count, dtype: int64
Keeping only these columns:  ['Item', 'Value', 'Year']
Year range before appending manual data:  1961 2022
Year range after appending manual data:  1961 2023
Year range before appending manual data:  1961 2023
Year range after appending manual data:  1961 2024
##############################################
---Final Years of data available for each crop---
Crop
Peanut     64
Corn       64
Millet     64


,Crop,Production,Year
0,Peanut,1728000.0,2024
1,Corn,929000.0,2024
2,Millet,1353000.0,2024
3,Rice,1521000.0,2024
4,Sorghum,445000.0,2024


### Vegetation Data Pre-processing

In [104]:
def load_vhi_data(vhi_data_filepath):

    # Load in vhi data
    vhi = pd.read_csv(vhi_data_filepath)

    # Print number of rows in vhi data
    print("Number of rows in VHI data: ", len(vhi))

    return vhi

vhi_data = load_vhi_data(vhi_data_filepath)

vhi_data.tail()

Number of rows in VHI data:  20468


,Indicator,Country,ADM1_CODE,Province,Date,Data,Year,Month,Dekad,Unit,Source
20463,Vegetation Health Index (VHI),Senegal,47589,Saint louis,2024-07-21,0.767,2024,7,3,,FAO-ASIS
20464,Vegetation Health Index (VHI),Senegal,1376,Sedhiou,2024-07-21,0.641,2024,7,3,,FAO-ASIS
20465,Vegetation Health Index (VHI),Senegal,1377,Tambacounda,2024-07-21,0.640,2024,7,3,,FAO-ASIS
20466,Vegetation Health Index (VHI),Senegal,2644,Thies,2024-07-21,0.644,2024,7,3,,FAO-ASIS
20467,Vegetation Health Index (VHI),Senegal,2645,Ziguinchor,2024-07-21,0.677,2024,7,3,,FAO-ASIS


In [105]:
def pivot_vhi_data_and_limit_to_latest_month(vhi_data, month):
    print(f"Filtering for only month {month} and pivoting province across the top and years down the side")

    # If month equals 1, filter for month = 1 but then set all values to 0
    # We do this so that we still have zeroed data
    if month == 1:
        vhi_filtered = vhi_data[vhi_data['Month'] == month].copy()
        vhi_filtered.loc[:, 'Data'] = 0
    else:
        # Filter for the latest month
        vhi_filtered = vhi_data[vhi_data['Month'] == month-1].copy()

    # Pivot the data
    vhi_filtered = vhi_filtered.pivot_table(index='Year', columns='Province', values='Data', aggfunc='mean')

    return vhi_filtered

# Get the average value of Data column, for July with Year as rows and Province as columns
vhi_pivot = pivot_vhi_data_and_limit_to_latest_month(vhi_data, month)

# View data
vhi_pivot.tail()

Filtering for only month 8 and pivoting province across the top and years down the side


Province,Dakar,Diourbel,Fatick,Kaffrine,Kaolack,Kedougou,Kolda,Louga,Matam,Saint louis,Sedhiou,Tambacounda,Thies,Ziguinchor
Year,,,,,,,,,,,,,,
2020,0.605333,0.694333,0.604333,0.667333,0.682000,0.655000,0.640667,0.729333,0.688333,0.791333,0.680667,0.709667,0.633333,0.680000
2021,0.582333,0.632667,0.570000,0.622333,0.582000,0.606000,0.543000,0.601000,0.480000,0.644667,0.574000,0.519333,0.600000,0.637000
2022,0.577250,0.722250,0.772000,0.747000,0.821250,0.666750,0.716000,0.605250,0.662500,0.624750,0.795500,0.789500,0.637750,0.760000
2023,0.590000,0.698000,0.660333,0.582667,0.664000,0.730333,0.643667,0.771667,0.779667,0.808333,0.724667,0.652000,0.693333,0.717333
2024,0.608000,0.687667,0.687667,0.625667,0.649333,0.629000,0.570667,0.732667,0.686667,0.820333,0.671667,0.536333,0.685000,0.717667


In [106]:
def pivot_vhi_data_and_cumulative_sum(vhi_data, month):
    print(f"Filtering for months less than or equal to {month} and calculating cumulative sum. Pivoting province across the top and years down the side.")

    # If month equals 1, filter for month = 1 but then set all values to 0
    # We do this so that we still have zeroed data
    if month == 1:
        vhi_filtered = vhi_data[vhi_data['Month'] == month].copy()
        vhi_filtered.loc[:, 'Data'] = 0
    else:
        # Filter for the latest month
        vhi_filtered = vhi_data[vhi_data['Month'] < month].copy()

    # Calculate the cumulative sum by year and province
    vhi_filtered['Cumulative_Data'] = vhi_filtered.groupby(['Year', 'Province'])['Data'].cumsum()

    # Pivot the data with the cumulative sum
    vhi_pivot = vhi_filtered.pivot_table(index='Year', columns='Province', values='Cumulative_Data', aggfunc='last')

    return vhi_pivot

# Get the cumulative sum of the Data column up to the specified month with Year as rows and Province as columns
vhi_pivot_cum = pivot_vhi_data_and_cumulative_sum(vhi_data, month)

# View data
vhi_pivot_cum.tail()

Filtering for months less than or equal to 8 and calculating cumulative sum. Pivoting province across the top and years down the side.


Province,Dakar,Diourbel,Fatick,Kaffrine,Kaolack,Kedougou,Kolda,Louga,Matam,Saint louis,Sedhiou,Tambacounda,Thies,Ziguinchor
Year,,,,,,,,,,,,,,
2020,10.886,11.203,10.371,10.952,10.595,11.382,11.641,12.573,12.081,12.790,11.257,11.803,11.102,10.715
2021,11.959,13.067,12.280,13.470,12.228,11.771,12.746,14.139,12.817,13.991,13.703,11.872,12.808,13.289
2022,12.607,14.188,14.354,14.181,14.708,13.486,13.595,14.352,14.766,14.297,16.040,14.364,13.836,15.151
2023,12.081,13.801,12.959,13.728,13.113,12.477,13.294,13.430,14.898,13.715,15.589,13.200,13.315,14.825
2024,11.456,11.640,11.954,11.173,11.584,10.507,10.381,12.011,12.314,12.629,12.449,11.120,12.112,12.960


In [107]:
def calculate_province_weighted_metrics_for_each_crop(data, province_splits, metric_name):

    print("Weighting the data using province splits for each crop")

    # Calculate the weighted average VHI for each crop
    for crop in province_splits['Crop']:
        # Extract the percentage values for the current crop (ignoring the Crop column)
        crop_percentages = province_splits.set_index('Crop').loc[crop]

        # Perform the sumproduct: multiply VHI values by the crop percentages and sum across provinces
        data[f'{crop}_Weighted'] = data.mul(crop_percentages).sum(axis=1)

    # Add the metric to all columns with _metric
    data = data.add_suffix(f'_{metric_name}')

    return data

# Calculate the weighted average VHI for each crop
vhi_pivot = calculate_province_weighted_metrics_for_each_crop(vhi_pivot, province_splits, "VHI")

vhi_pivot.tail()

Weighting the data using province splits for each crop


Province,Dakar_VHI,Diourbel_VHI,Fatick_VHI,Kaffrine_VHI,Kaolack_VHI,Kedougou_VHI,Kolda_VHI,Louga_VHI,Matam_VHI,Saint louis_VHI,Sedhiou_VHI,Tambacounda_VHI,Thies_VHI,Ziguinchor_VHI,Corn_Weighted_VHI,Cotton_Weighted_VHI,Millet_Weighted_VHI,Peanut_Weighted_VHI,Rice_Weighted_VHI,Sorghum_Weighted_VHI
Year,,,,,,,,,,,,,,,,,,,,
2020,0.605333,0.694333,0.604333,0.667333,0.682000,0.655000,0.640667,0.729333,0.688333,0.791333,0.680667,0.709667,0.633333,0.680000,0.667743,0.660923,0.662250,0.664297,0.692730,0.675383
2021,0.582333,0.632667,0.570000,0.622333,0.582000,0.606000,0.543000,0.601000,0.480000,0.644667,0.574000,0.519333,0.600000,0.637000,0.564600,0.545530,0.587613,0.582376,0.591813,0.577257
2022,0.577250,0.722250,0.772000,0.747000,0.821250,0.666750,0.716000,0.605250,0.662500,0.624750,0.795500,0.789500,0.637750,0.760000,0.756859,0.734818,0.748883,0.733505,0.721255,0.750157
2023,0.590000,0.698000,0.660333,0.582667,0.664000,0.730333,0.643667,0.771667,0.779667,0.808333,0.724667,0.652000,0.693333,0.717333,0.676961,0.648913,0.657861,0.658291,0.715783,0.655110
2024,0.608000,0.687667,0.687667,0.625667,0.649333,0.629000,0.570667,0.732667,0.686667,0.820333,0.671667,0.536333,0.685000,0.717667,0.630485,0.570130,0.649637,0.641322,0.682577,0.623287


In [108]:
# Define function for VHI data preprocessing
def process_vhi_data(vhi_data_filepath, province_splits, month):
    # Print status
    print("######## Processing VHI Data ##########")

    # Load in vhi data
    vhi_data = load_vhi_data(vhi_data_filepath)

    # Filter for the latest month
    vhi_pivot = pivot_vhi_data_and_limit_to_latest_month(vhi_data, month)
    vhi_pivot_cum = pivot_vhi_data_and_cumulative_sum(vhi_data, month)

    # Calculate the weighted average VHI for each crop
    vhi_pivot = calculate_province_weighted_metrics_for_each_crop(vhi_pivot, province_splits, "VHI")
    vhi_pivot_cum = calculate_province_weighted_metrics_for_each_crop(vhi_pivot_cum, province_splits, "VHI_Cumulative")

    print("#######################################")

    return vhi_pivot, vhi_pivot_cum

# Get data for current month to keep notebook going
vhi_data, vhi_cumulative = process_vhi_data(vhi_data_filepath, province_splits, month)

# View data
vhi_data.tail()

######## Processing VHI Data ##########
Number of rows in VHI data:  20468
Filtering for only month 8 and pivoting province across the top and years down the side
Filtering for months less than or equal to 8 and calculating cumulative sum. Pivoting province across the top and years down the side.
Weighting the data using province splits for each crop
Weighting the data using province splits for each crop
#######################################


Province,Dakar_VHI,Diourbel_VHI,Fatick_VHI,Kaffrine_VHI,Kaolack_VHI,Kedougou_VHI,Kolda_VHI,Louga_VHI,Matam_VHI,Saint louis_VHI,Sedhiou_VHI,Tambacounda_VHI,Thies_VHI,Ziguinchor_VHI,Corn_Weighted_VHI,Cotton_Weighted_VHI,Millet_Weighted_VHI,Peanut_Weighted_VHI,Rice_Weighted_VHI,Sorghum_Weighted_VHI
Year,,,,,,,,,,,,,,,,,,,,
2020,0.605333,0.694333,0.604333,0.667333,0.682000,0.655000,0.640667,0.729333,0.688333,0.791333,0.680667,0.709667,0.633333,0.680000,0.667743,0.660923,0.662250,0.664297,0.692730,0.675383
2021,0.582333,0.632667,0.570000,0.622333,0.582000,0.606000,0.543000,0.601000,0.480000,0.644667,0.574000,0.519333,0.600000,0.637000,0.564600,0.545530,0.587613,0.582376,0.591813,0.577257
2022,0.577250,0.722250,0.772000,0.747000,0.821250,0.666750,0.716000,0.605250,0.662500,0.624750,0.795500,0.789500,0.637750,0.760000,0.756859,0.734818,0.748883,0.733505,0.721255,0.750157
2023,0.590000,0.698000,0.660333,0.582667,0.664000,0.730333,0.643667,0.771667,0.779667,0.808333,0.724667,0.652000,0.693333,0.717333,0.676961,0.648913,0.657861,0.658291,0.715783,0.655110
2024,0.608000,0.687667,0.687667,0.625667,0.649333,0.629000,0.570667,0.732667,0.686667,0.820333,0.671667,0.536333,0.685000,0.717667,0.630485,0.570130,0.649637,0.641322,0.682577,0.623287


In [109]:
vhi_cumulative.tail()

Province,Dakar_VHI_Cumulative,Diourbel_VHI_Cumulative,Fatick_VHI_Cumulative,Kaffrine_VHI_Cumulative,Kaolack_VHI_Cumulative,Kedougou_VHI_Cumulative,Kolda_VHI_Cumulative,Louga_VHI_Cumulative,Matam_VHI_Cumulative,Saint louis_VHI_Cumulative,Sedhiou_VHI_Cumulative,Tambacounda_VHI_Cumulative,Thies_VHI_Cumulative,Ziguinchor_VHI_Cumulative,Corn_Weighted_VHI_Cumulative,Cotton_Weighted_VHI_Cumulative,Millet_Weighted_VHI_Cumulative,Peanut_Weighted_VHI_Cumulative,Rice_Weighted_VHI_Cumulative,Sorghum_Weighted_VHI_Cumulative
Year,,,,,,,,,,,,,,,,,,,,
2020,10.886,11.203,10.371,10.952,10.595,11.382,11.641,12.573,12.081,12.790,11.257,11.803,11.102,10.715,11.267771,11.61721,11.037688,11.197157,11.61500,11.29110
2021,11.959,13.067,12.280,13.470,12.228,11.771,12.746,14.139,12.817,13.991,13.703,11.872,12.808,13.289,12.750039,12.50959,12.867154,12.843207,13.33554,12.88123
2022,12.607,14.188,14.354,14.181,14.708,13.486,13.595,14.352,14.766,14.297,16.040,14.364,13.836,15.151,14.542560,13.84559,14.389524,14.205186,14.65857,14.41201
2023,12.081,13.801,12.959,13.728,13.113,12.477,13.294,13.430,14.898,13.715,15.589,13.200,13.315,14.825,13.747919,13.26432,13.529542,13.392561,14.20595,13.70640
2024,11.456,11.640,11.954,11.173,11.584,10.507,10.381,12.011,12.314,12.629,12.449,11.120,12.112,12.960,11.498872,10.64562,11.572592,11.420863,11.92581,11.39680


In [110]:
def process_vhi_data_across_months():
    # Process production data across months
    for month in range(1, 13):
        # Process production data
        vhi_data, vhi_cumulative = process_vhi_data(vhi_data_filepath, province_splits, month)

        # Save processed data in data folder
        vhi_data.to_csv(f"processed_data/vhi_data/vhi_data_{month}.csv", index=True)

        # Save processed data in data folder
        vhi_cumulative.to_csv(f"processed_data/vhi_cumulative/vhi_cumulative_{month}.csv", index=True)

process_vhi_data_across_months()

######## Processing VHI Data ##########
Number of rows in VHI data:  20468
Filtering for only month 1 and pivoting province across the top and years down the side
Filtering for months less than or equal to 1 and calculating cumulative sum. Pivoting province across the top and years down the side.
Weighting the data using province splits for each crop
Weighting the data using province splits for each crop
#######################################
######## Processing VHI Data ##########
Number of rows in VHI data:  20468
Filtering for only month 2 and pivoting province across the top and years down the side
Filtering for months less than or equal to 2 and calculating cumulative sum. Pivoting province across the top and years down the side.
Weighting the data using province splits for each crop
Weighting the data using province splits for each crop
#######################################
######## Processing VHI Data ##########
Number of rows in VHI data:  20468
Filtering for only month 3 an

### Rainfall Data Pre-processing

In [111]:
def load_clean_and_filter_rainfall_data(rainfall_data_filepath, month):

    # Load in rainfall data
    rainfall = pd.read_csv(rainfall_data_filepath)

    # Convert the 'Date' column to datetime format
    rainfall['Date'] = pd.to_datetime(rainfall['Date'], format='%Y-%m-%d')

    # If month equals 1, filter for month = 1 but then set all values to 0
    if month == 1:
        rainfall_filtered = rainfall[rainfall['Date'].dt.month == month].copy()
        rainfall_filtered.loc[:, 'Data'] = 0
    else:
        # Filter out rows where the month is August (8) or higher
        rainfall_filtered = rainfall[rainfall['Date'].dt.month <= (month-1)].copy()

    return rainfall_filtered

rainfall_data = load_clean_and_filter_rainfall_data(rainfall_data_filepath, month)
rainfall_data.tail()

,Indicator,Country,ADM1_CODE,Province,Land_Type,Date,Data,Data_long_term_Average,Year,Month,Dekad,Unit,Source
14695,RS-Based Estimated Precipitation,Senegal,47589,Saint louis,Crop Area,2024-07-21,20.731,19.363,2024,7,3,MM,FEWSNet RFE
14696,RS-Based Estimated Precipitation,Senegal,1376,Sedhiou,Crop Area,2024-07-21,73.419,75.158,2024,7,3,MM,FEWSNet RFE
14697,RS-Based Estimated Precipitation,Senegal,1377,Tambacounda,Crop Area,2024-07-21,41.347,41.671,2024,7,3,MM,FEWSNet RFE
14698,RS-Based Estimated Precipitation,Senegal,2644,Thies,Crop Area,2024-07-21,24.401,31.262,2024,7,3,MM,FEWSNet RFE
14699,RS-Based Estimated Precipitation,Senegal,2645,Ziguinchor,Crop Area,2024-07-21,96.209,90.916,2024,7,3,MM,FEWSNet RFE


In [112]:
def cumulative_rainfall_sum_and_pivot(rainfall_data):

    # Group by Year and Province, then sum the 'Data' column
    rainfall_grouped = rainfall_data.groupby(['Year', 'Province'])['Data'].sum().reset_index()

    # Pivot the table to have years as rows and provinces as columns
    rainfall_pivot = rainfall_grouped.pivot(index='Year', columns='Province', values='Data')

    return rainfall_pivot

rainfall_pivot = cumulative_rainfall_sum_and_pivot(rainfall_data)
rainfall_pivot.tail()

Province,Dakar,Diourbel,Fatick,Kaffrine,Kaolack,Kedougou,Kolda,Louga,Matam,Saint louis,Sedhiou,Tambacounda,Thies,Ziguinchor
Year,,,,,,,,,,,,,,
2020,251.537,200.120,281.237,263.514,308.791,609.945,395.749,179.091,214.231,117.752,524.200,312.050,214.184,558.237
2021,111.317,213.947,192.499,341.560,327.854,686.113,385.281,143.155,206.965,106.635,393.273,323.653,146.847,329.645
2022,267.542,237.950,222.557,305.124,231.658,710.692,387.840,143.657,158.623,178.726,541.975,357.842,203.556,773.699
2023,84.889,141.247,169.778,184.895,187.370,488.881,301.353,123.207,180.216,134.441,292.405,248.395,123.378,292.896
2024,71.441,114.043,175.167,171.520,202.497,404.991,264.203,90.903,148.701,83.387,327.911,196.450,81.128,411.035


In [113]:
rainfall_data = calculate_province_weighted_metrics_for_each_crop(rainfall_pivot, province_splits, "Rainfall")

rainfall_data.tail()

Weighting the data using province splits for each crop


Province,Dakar_Rainfall,Diourbel_Rainfall,Fatick_Rainfall,Kaffrine_Rainfall,Kaolack_Rainfall,Kedougou_Rainfall,Kolda_Rainfall,Louga_Rainfall,Matam_Rainfall,Saint louis_Rainfall,Sedhiou_Rainfall,Tambacounda_Rainfall,Thies_Rainfall,Ziguinchor_Rainfall,Corn_Weighted_Rainfall,Cotton_Weighted_Rainfall,Millet_Weighted_Rainfall,Peanut_Weighted_Rainfall,Rice_Weighted_Rainfall,Sorghum_Weighted_Rainfall
Year,,,,,,,,,,,,,,,,,,,,
2020,251.537,200.120,281.237,263.514,308.791,609.945,395.749,179.091,214.231,117.752,524.200,312.050,214.184,558.237,369.533827,381.47919,296.557434,298.202448,381.47259,335.56418
2021,111.317,213.947,192.499,341.560,327.854,686.113,385.281,143.155,206.965,106.635,393.273,323.653,146.847,329.645,329.479472,385.24352,284.131910,291.412548,305.48612,331.55065
2022,267.542,237.950,222.557,305.124,231.658,710.692,387.840,143.657,158.623,178.726,541.975,357.842,203.556,773.699,365.706906,395.55535,287.462174,292.619149,431.71412,352.22055
2023,84.889,141.247,169.778,184.895,187.370,488.881,301.353,123.207,180.216,134.441,292.405,248.395,123.378,292.896,244.161232,292.31297,193.749956,203.342399,250.22950,228.90481
2024,71.441,114.043,175.167,171.520,202.497,404.991,264.203,90.903,148.701,83.387,327.911,196.450,81.128,411.035,235.762955,251.09790,183.792810,187.035869,253.99171,214.14117


In [114]:
# Process all rainfall data function
def process_rainfall_data(rainfall_data_filepath, month, province_splits):
    # Print status
    print("######## Processing Rainfall Data ##########")

    # Load in rainfall data
    rainfall_data = load_clean_and_filter_rainfall_data(rainfall_data_filepath, month)

    # Group by Year and Province, then sum the 'Data' column
    rainfall_pivot = cumulative_rainfall_sum_and_pivot(rainfall_data)

    # Calculate the weighted average Rainfall for each crop
    rainfall_pivot = calculate_province_weighted_metrics_for_each_crop(rainfall_pivot, province_splits, "Rainfall")

    print("#############################################")

    return rainfall_pivot

rainfall_data = process_rainfall_data(rainfall_data_filepath, month, province_splits)

rainfall_data.tail()

######## Processing Rainfall Data ##########
Weighting the data using province splits for each crop
#############################################


Province,Dakar_Rainfall,Diourbel_Rainfall,Fatick_Rainfall,Kaffrine_Rainfall,Kaolack_Rainfall,Kedougou_Rainfall,Kolda_Rainfall,Louga_Rainfall,Matam_Rainfall,Saint louis_Rainfall,Sedhiou_Rainfall,Tambacounda_Rainfall,Thies_Rainfall,Ziguinchor_Rainfall,Corn_Weighted_Rainfall,Cotton_Weighted_Rainfall,Millet_Weighted_Rainfall,Peanut_Weighted_Rainfall,Rice_Weighted_Rainfall,Sorghum_Weighted_Rainfall
Year,,,,,,,,,,,,,,,,,,,,
2020,251.537,200.120,281.237,263.514,308.791,609.945,395.749,179.091,214.231,117.752,524.200,312.050,214.184,558.237,369.533827,381.47919,296.557434,298.202448,381.47259,335.56418
2021,111.317,213.947,192.499,341.560,327.854,686.113,385.281,143.155,206.965,106.635,393.273,323.653,146.847,329.645,329.479472,385.24352,284.131910,291.412548,305.48612,331.55065
2022,267.542,237.950,222.557,305.124,231.658,710.692,387.840,143.657,158.623,178.726,541.975,357.842,203.556,773.699,365.706906,395.55535,287.462174,292.619149,431.71412,352.22055
2023,84.889,141.247,169.778,184.895,187.370,488.881,301.353,123.207,180.216,134.441,292.405,248.395,123.378,292.896,244.161232,292.31297,193.749956,203.342399,250.22950,228.90481
2024,71.441,114.043,175.167,171.520,202.497,404.991,264.203,90.903,148.701,83.387,327.911,196.450,81.128,411.035,235.762955,251.09790,183.792810,187.035869,253.99171,214.14117


In [115]:
def process_rainfall_data_across_months():
    # Process rainfall data across months
    for month in range(1, 13):
        # Process production data
        rainfall_data = process_rainfall_data(rainfall_data_filepath, month, province_splits)

        # Save processed data in data folder
        rainfall_data.to_csv(f"processed_data/rainfall_data/rainfall_data_{month}.csv", index=True)

process_rainfall_data_across_months()

######## Processing Rainfall Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Rainfall Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Rainfall Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Rainfall Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Rainfall Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Rainfall Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Rainfall Data ##########
Weighting the data using province splits for each crop
########################

### Processing temperature data 

In [116]:
def load_temp_data(temp_data_filepath):

    # Load in rainfall data
    temp = pd.read_csv(temp_data_filepath)

    # Convert Month column from string to number of the month
    temp['Month'] = pd.to_datetime(temp['Month'], format='%B').dt.month

    return temp

temp_data = load_temp_data(temp_data_filepath)
temp_data.head()

,Year,Month,Dakar,Diourbel,Fatick,Kaffrine,Kaolack,Kedougou,Kolda,Louga,Matam,Saint louis,Sedhiou,Tambacounda,Thies,Ziguinchor
0,1973,1,19.863158,24.665000,26.477778,26.477778,26.477778,29.587500,25.488889,22.586957,24.250000,22.586957,25.488889,27.175000,24.665000,25.084211
1,1973,2,19.528000,25.528000,27.022222,27.022222,27.022222,30.811765,27.307143,21.839130,27.350000,21.839130,27.307143,29.179167,25.528000,25.296154
2,1973,3,20.392000,27.370833,28.638889,28.638889,28.638889,31.933333,29.372222,23.540741,30.127273,23.540741,29.372222,31.290909,27.370833,26.383333
3,1973,4,21.391667,28.176000,29.306667,29.306667,29.306667,33.706667,31.381818,23.608333,34.590000,23.608333,31.381818,32.579167,28.176000,26.396000
4,1973,5,23.535714,28.411111,29.382609,29.382609,29.382609,32.122727,30.835000,24.056667,34.833333,24.056667,30.835000,33.103448,28.411111,27.241379


In [117]:
def cumulative_temperature_up_to_month(temp_data, month):

    # Filter out where Month is >= month
    if month == 1:
        temp_data = temp_data[temp_data['Month'] <= month]
    else:
        temp_data = temp_data[temp_data['Month'] < month]

    # Average the values of every column and aggregating them to the Year level
    temp_data = temp_data.groupby('Year').mean()

    # Remove the Month column from temp_data
    temp_data = temp_data.drop(columns='Month')

    return temp_data

temp_data_agg = cumulative_temperature_up_to_month(temp_data, month)
temp_data_agg.tail()

,Dakar,Diourbel,Fatick,Kaffrine,Kaolack,Kedougou,Kolda,Louga,Matam,Saint louis,Sedhiou,Tambacounda,Thies,Ziguinchor
Year,,,,,,,,,,,,,,
2020,24.421942,29.581475,30.155426,30.155426,30.155426,31.191928,30.091250,25.941894,32.163808,25.941894,30.091250,32.089809,29.581475,28.052505
2021,23.057678,28.671844,29.297449,29.297449,29.297449,31.424126,29.705776,24.592623,32.326344,24.592623,29.705776,31.618769,28.671844,27.684270
2022,24.078084,28.618615,29.095099,29.095099,29.095099,30.737452,29.920214,25.190055,31.640482,25.190055,29.920214,31.374305,28.618615,27.386487
2023,24.427537,29.441127,29.657441,29.657441,29.657441,31.428165,30.094896,25.780730,32.222281,25.780730,30.094896,32.055571,29.441127,27.694789
2024,24.602554,28.912719,29.799154,29.799154,29.799154,32.085397,30.491683,25.490691,32.470683,25.490691,30.491683,32.421171,28.912719,27.918506


In [118]:
# Process all rainfall data function
def process_temp_data(temp_data_filepath, month, province_splits):
    # Print status
    print("######## Processing Temperature Data ##########")

    # Load in rainfall data
    temp_data = load_temp_data(temp_data_filepath)

    # Group by Year and Province, then sum the 'Data' column
    temp_pivot = cumulative_temperature_up_to_month(temp_data, month)

    # Calculate the weighted average Rainfall for each crop
    temp_pivot = calculate_province_weighted_metrics_for_each_crop(temp_pivot, province_splits, "Temperature")

    print("#############################################")

    return temp_pivot

temp_data = process_temp_data(temp_data_filepath, month, province_splits)

temp_data.tail()

######## Processing Temperature Data ##########
Weighting the data using province splits for each crop
#############################################


,Dakar_Temperature,Diourbel_Temperature,Fatick_Temperature,Kaffrine_Temperature,Kaolack_Temperature,Kedougou_Temperature,Kolda_Temperature,Louga_Temperature,Matam_Temperature,Saint louis_Temperature,Sedhiou_Temperature,Tambacounda_Temperature,Thies_Temperature,Ziguinchor_Temperature,Corn_Weighted_Temperature,Cotton_Weighted_Temperature,Millet_Weighted_Temperature,Peanut_Weighted_Temperature,Rice_Weighted_Temperature,Sorghum_Weighted_Temperature
Year,,,,,,,,,,,,,,,,,,,,
2020,24.421942,29.581475,30.155426,30.155426,30.155426,31.191928,30.091250,25.941894,32.163808,25.941894,30.091250,32.089809,29.581475,28.052505,30.122612,30.660781,29.946441,29.751128,28.797453,30.178972
2021,23.057678,28.671844,29.297449,29.297449,29.297449,31.424126,29.705776,24.592623,32.326344,24.592623,29.705776,31.618769,28.671844,27.684270,29.545488,30.262625,29.158174,28.989444,28.169420,29.539240
2022,24.078084,28.618615,29.095099,29.095099,29.095099,30.737452,29.920214,25.190055,31.640482,25.190055,29.920214,31.374305,28.618615,27.386487,29.561792,30.283264,29.084259,28.953388,28.357590,29.487577
2023,24.427537,29.441127,29.657441,29.657441,29.657441,31.428165,30.094896,25.780730,32.222281,25.780730,30.094896,32.055571,29.441127,27.694789,29.969732,30.638813,29.631572,29.472058,28.689961,29.966750
2024,24.602554,28.912719,29.799154,29.799154,29.799154,32.085397,30.491683,25.490691,32.470683,25.490691,30.491683,32.421171,28.912719,27.918506,30.229267,31.028126,29.706035,29.589640,28.873522,30.188620


In [120]:
def process_temp_data_across_months():
    # Process rainfall data across months
    for month in range(1, 13):
        # Process production data
        temp_data = process_temp_data(temp_data_filepath, month, province_splits)

        # Save processed data in data folder
        temp_data.to_csv(f"processed_data/temp_data/temp_data_{month}.csv", index=True)

process_temp_data_across_months()

######## Processing Temperature Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Temperature Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Temperature Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Temperature Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Temperature Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Temperature Data ##########
Weighting the data using province splits for each crop
#############################################
######## Processing Temperature Data ##########
Weighting the data using province splits for each crop
###

### Merging all datasets together

In [ ]:
# For each month, get the rainfall data for right month, vhi data for right month, and production data (same across), and merge all together on Year
# Then save it as final_data_monthnum.csv in the processed_data/final_data folder

def merge_and_save_final_data(month):
    # Load in the data
    production_data = pd.read_csv("processed_data/production_data.csv")
    vhi_data = pd.read_csv(f"processed_data/vhi_data/vhi_data_{month}.csv")
    vhi_cumulative = pd.read_csv(f"processed_data/vhi_cumulative/vhi_cumulative_{month}.csv")
    rainfall_data = pd.read_csv(f"processed_data/rainfall_data/rainfall_data_{month}.csv")
    temp_data = pd.read_csv(f"processed_data/temp_data/temp_data_{month}.csv")

    # Merge the data together on Year
    final_data = production_data.merge(vhi_data, on='Year').merge(vhi_cumulative, on='Year').merge(rainfall_data, on='Year').merge(temp_data, on='Year')

    # Save the data
    final_data.to_csv(f"processed_data/final_data/final_data_{month}.csv", index=False)

    return final_data

final_data = merge_and_save_final_data(month)
final_data.columns

Index(['Crop', 'Production', 'Year', 'Dakar_VHI', 'Diourbel_VHI', 'Fatick_VHI',
       'Kaffrine_VHI', 'Kaolack_VHI', 'Kedougou_VHI', 'Kolda_VHI', 'Louga_VHI',
       'Matam_VHI', 'Saint louis_VHI', 'Sedhiou_VHI', 'Tambacounda_VHI',
       'Thies_VHI', 'Ziguinchor_VHI', 'Corn_Weighted_VHI',
       'Cotton_Weighted_VHI', 'Millet_Weighted_VHI', 'Peanut_Weighted_VHI',
       'Rice_Weighted_VHI', 'Sorghum_Weighted_VHI', 'Dakar_VHI_Cumulative',
       'Diourbel_VHI_Cumulative', 'Fatick_VHI_Cumulative',
       'Kaffrine_VHI_Cumulative', 'Kaolack_VHI_Cumulative',
       'Kedougou_VHI_Cumulative', 'Kolda_VHI_Cumulative',
       'Louga_VHI_Cumulative', 'Matam_VHI_Cumulative',
       'Saint louis_VHI_Cumulative', 'Sedhiou_VHI_Cumulative',
       'Tambacounda_VHI_Cumulative', 'Thies_VHI_Cumulative',
       'Ziguinchor_VHI_Cumulative', 'Corn_Weighted_VHI_Cumulative',
       'Cotton_Weighted_VHI_Cumulative', 'Millet_Weighted_VHI_Cumulative',
       'Peanut_Weighted_VHI_Cumulative', 'Rice_Weighted_V

In [121]:
# Merge and save final data for each month of the year
for month in range(1, 13):
    final_data = merge_and_save_final_data(month)